**Llama 2**

The Llama 2 is a collection of pretrained and fine-tuned generative text models, ranging from 7 billion to 70 billion parameters, designed for dialogue use cases.

 It outperforms open-source chat models on most benchmarks and is on par with popular closed-source models in human evaluations for helpfulness and safety.

[Llama 2 13B-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat)

`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage.

#  Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).



In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

#**Step 1: Install All the Required Packages**

In [1]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

#**Step 2: Import All the Required Libraries**

In [2]:
import os

In [3]:
from huggingface_hub import hf_hub_download

In [4]:
from llama_cpp import Llama

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3060, compute capability 8.6


#**Step 3: Download the Model**

In [5]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

#**Step 4: Loading the Model**

In [6]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=os.cpu_count(), # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=-1 # Change this value based on your model and your GPU VRAM pool.
    )

llama.cpp: loading model from /home/xodiec/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_inte

In [7]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

2147483647

#**Step 5: Create a Prompt Template**

In [12]:
prompt = "I would like you to tell me the following informations related to the movie 'Sleepless in Seattle' (1993): Genre, Brief description of the plot, Director, Principal 4/6 members of the cast."
prompt_template=f'''SYSTEM: You are a helpful assistant who's expert in the field of Cinema. For the requested movie, answer as follows:
- Genre: use as least tokens as possible for this information
- Description of the plot: report the main details of the movie's plot
- Director: only report the director's name and surname
- Principal 4/6 Members of the Cast: report names and surnames

After you generated all these informations, end your response with the special token ['END'].

USER: {prompt}

ASSISTANT:
'''

#**Step 6: Generating the Response**

In [13]:
response=lcpp_llm(prompt=prompt_template, max_tokens=8192, temperature=0.1, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  1412.42 ms
llama_print_timings:      sample time =    71.76 ms /   197 runs   (    0.36 ms per token,  2745.34 tokens per second)
llama_print_timings: prompt eval time =  1199.31 ms /   134 tokens (    8.95 ms per token,   111.73 tokens per second)
llama_print_timings:        eval time = 11136.98 ms /   196 runs   (   56.82 ms per token,    17.60 tokens per second)
llama_print_timings:       total time = 12748.67 ms


In [14]:
print(response)

{'id': 'cmpl-b7892758-bfc9-49fb-89c3-9ee8775d0fb7', 'object': 'text_completion', 'created': 1713271041, 'model': '/home/xodiec/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful assistant who's expert in the field of Cinema. For the requested movie, answer as follows:\n- Genre: use as least tokens as possible for this information\n- Description of the plot: report the main details of the movie's plot\n- Director: only report the director's name and surname\n- Principal 4/6 Members of the Cast: report names and surnames\n\nAfter you generated all these informations, end your response with the special token ['END'].\n\nUSER: I would like you to tell me the following informations related to the movie 'Sleepless in Seattle' (1993): Genre, Brief description of the plot, Director, Principal 4/6 members of the cast.\n\nASSISTANT:\n\nGenre: Romantic 

In [15]:
print(response["choices"][0]["text"])

SYSTEM: You are a helpful assistant who's expert in the field of Cinema. For the requested movie, answer as follows:
- Genre: use as least tokens as possible for this information
- Description of the plot: report the main details of the movie's plot
- Director: only report the director's name and surname
- Principal 4/6 Members of the Cast: report names and surnames

After you generated all these informations, end your response with the special token ['END'].

USER: I would like you to tell me the following informations related to the movie 'Sleepless in Seattle' (1993): Genre, Brief description of the plot, Director, Principal 4/6 members of the cast.

ASSISTANT:

Genre: Romantic Comedy [Rom-Com]

Description of the Plot: Sam Baldwin (Tom Hanks) is a recently widowed father who, still grieving the loss of his wife, decides to move to Seattle with his son Jonah. During a radio talk show, he shares his story and captures the attention of Annie Reed (Meg Ryan), a reporter from Baltimore 